In [ ]:
import re
import os
print(os.getcwd())

In [ ]:
path_prepend = 'tex_tables'

In [61]:
pubs_list = ['AER', 'ECA', 'JPE', 'QJE', 'RES']
# pubs_list = ['AER']
pub_full_name_dict = {
    'AER' : 'American Economic Review',
    'ECA' : 'Econometrica',
    'JPE' : 'Journal of Political Economy',
    'QJE' : 'Quarterly Journal of Economics',
    'RES' : 'Review of Economic Studies'
}
print(pubs_list)
for pub in pubs_list:
    with open(path_prepend + '/jel_ranking_table_' + pub + '.tex', 'r') as pub_tab:
        old_tex = pub_tab.read()
        pub_full_name  = pub_full_name_dict.get(pub)

        # CHANGING ALIGNMENT OF COLUMNS
        new_tex = old_tex.replace('ccccccc', 'lllllll')
        
        # CHANING POSITIONING OF TABLE WITHIN PAGES
        new_tex = re.sub(r'!htbp', 'h', new_tex)

        # ADDING A PUBLICATION-LEVEL HEADER 
        new_tex = re.sub(r'\\hline \n\\hline \\\\\[-1.8ex\] ', 
            r'\\hline \n\\hline \\\\[-1.8ex] \\multicolumn{7}{c}{Publication : \\textit{' + pub_full_name + r'}} \\\\\n \\cline{1-7} \\\\',
            new_tex)

        # CLEANING UP THE COLUMN HEADERS TO BE ONLY YEARS (removing underscores and year so that 'AER\_1991' becomes '1991)
        new_tex = re.sub(r'[A-Z]{3}\\_', '', new_tex)

        # WRAPPING THE ENTIRE TABLE IN AN ENVIRONMENT THAT SCALES THE TABLE TO THE TEXT-WIDTH OF THE PAGE
        # new_tex = re.sub(r'\\label{} \n', r'\\label{} \n\\begin{adjustbox}{max width = \\textwidth}\n', new_tex)
        # new_tex = re.sub(r'\\end{tabular} \n', r'\\end{tabular} \n\\end{adjustbox}\n', new_tex)

        # CREATE A NICE, CLEANED UP PUBLICATION LEVEL TABLE (STORED AS TEMP, WILL BE DELETED LATER)
        with open(path_prepend + '/jel_ranking_table_' + pub +'_temp.tex', 'w') as temp_output:
            temp_output.write(new_tex)

first_tex_str = open(path_prepend + '/jel_ranking_table_' + pubs_list[0] + '_temp.tex', 'r').read()

for i in range(1, len(pubs_list)): 
    print('EXTRACING TEX FOR PUBLICATION: {}'.format(pubs_list[i]))
    with open(path_prepend + '/jel_ranking_table_' + pubs_list[i] + '_temp.tex', 'r') as temp_table_file:
        temp_table_tex = temp_table_file.read()

        table_body_of_interest = re.search(r'(?s)(?<=lllllll} \n\\\\\[-1.8ex\]\\hline).*(?=\\hline)', temp_table_tex).group()
        table_body_of_interest = re.sub(r'\\', r'\\\\', table_body_of_interest)
        # print(type(table_body_of_interest))
        # print(table_body_of_interest)


    if i == 1:
        complete_table_tex = re.sub(r'\\hline \\\\\[-1.8ex\] \n\\end{tabular} \n\\end{adjustbox} \n\\end{table}', table_body_of_interest, first_tex_str)
    else:
        complete_table_tex = re.sub(r'\\end{tabular}', table_body_of_interest + r'\n\\end{tabular}', complete_table_tex)

    # complete_table_tex = complete_table_tex + r'\hline + \\[-1.8ex] \n\\end{tabular} \n\\end{adjustbox} \n\\end{table}'

with open(path_prepend + '/jel_ranking_table_together.tex', 'w') as final_output:
    final_output.write(complete_table_tex)


['AER', 'ECA', 'JPE', 'QJE', 'RES']
EXTRACING TEX FOR PUBLICATION: ECA
EXTRACING TEX FOR PUBLICATION: JPE
EXTRACING TEX FOR PUBLICATION: QJE
EXTRACING TEX FOR PUBLICATION: RES


In [55]:
test_string = '\hline [] \endline'
re.sub(r'\[', r'\[', test_string)

'\\hline \\[] \\endline'